In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/output

In [ ]:
import pandas as pd
import glob
import os

folder_path = "/content/drive/MyDrive/output"

csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = []

for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)

print(combined_df.shape)

In [ ]:
combined_df_sorted = combined_df.sort_values(by='idx', ascending=True)
combined_df_sorted['keywords'] = combined_df_sorted['keywords'].str.replace(', ', ',', regex=False)

print(combined_df_sorted)

In [ ]:
file_path_ner = "/content/drive/MyDrive/NER/output_ner.csv"
df_ner = pd.read_csv(file_path_ner, encoding='utf-8-sig')

In [ ]:
df_ner = df_ner[(df_ner['idx'] >= 0) & (df_ner['idx'] <= 43902)]
print(df_ner)

In [ ]:
import pandas as pd

idx_values = list(range(43903))

df = pd.DataFrame({
    'idx': idx_values,
    'tags': [''] * len(idx_values)
})

print(df.info())

In [ ]:
import pandas as pd

combined_keywords = pd.concat([df_ner[['idx', 'tags']], combined_df[['idx', 'tags']]])
combined_keywords = combined_keywords.drop_duplicates(subset='idx', keep='last')
combined_keywords_sorted = combined_keywords.sort_values(by='idx', ascending=True)

print(combined_keywords.head())

In [ ]:
import pandas as pd

combined_keywords_grouped = combined_keywords.groupby('idx', as_index=False).agg({
    'tags': lambda x: ','.join(filter(None, map(str, x)))
})
combined_keywords_grouped['tags'] = combined_keywords_grouped['tags'].fillna('').str.replace('_', ' ', regex=False)
combined_keywords_grouped['tags'] = combined_keywords_grouped['tags'].fillna('').str.lower()
print(combined_keywords_grouped)

In [ ]:
def remove_duplicate_keywords(keywords):
    if pd.isna(keywords):
        return keywords
    keywords_list = keywords.split(',')
    unique_keywords = list(dict.fromkeys(keywords_list))
    return ','.join(unique_keywords)


combined_keywords_grouped['tags'] = combined_keywords_grouped['tags'].apply(remove_duplicate_keywords)

print(combined_keywords_grouped)


In [ ]:
merged_df = pd.merge(df, combined_keywords_grouped, on='idx', how='left')
merged_df = merged_df.T.drop_duplicates().T

print(merged_df)

In [ ]:
import numpy as np

merged_df['tags'] = merged_df['tags_x'].fillna('') + ', ' + merged_df['tags_y'].fillna('')
merged_df['tags'] = merged_df['tags'].str.strip(', ')
merged_df['tags'] = merged_df['tags'].replace('', np.nan)

merged_df.drop(columns=['tags_x', 'tags_y'], inplace=True)

print(merged_df)

In [ ]:
df_test = pd.read_csv('/content/articles_testing.tsv', sep='\t')

combined_df = pd.concat([df_test, merged_df[['tags']]], axis=1)

print(combined_df)

In [ ]:
combined_df.to_csv('nhom2_sol1.tsv', sep='\t', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('nhom2_sol1.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>